# Build a Multi-User Conversational Tool-Calling Agentic AI Research Assistant with LangChain

## 🎯 Learning Objectives

By the end of this notebook, you will understand:
1. **Multi-User Session Management**: How to store and retrieve conversation history per user session
2. **SQL-based Memory Storage**: Using `SQLChatMessageHistory` to persist conversations in a database
3. **Conversational AI Agents**: Building agents that maintain context across multiple interactions
4. **Tool Integration**: Combining web search and weather tools with session-based memory

## 📋 Prerequisites
- Completion of M2 notebook (Tool-Calling Agentic AI Research Assistant)
- Basic understanding of LangChain agents and tools
- API keys: OpenAI, Tavily Search, WeatherAPI (OpenWeatherMap)

## 📚 Conceptual Overview

This demo builds on the previous Tool-Calling Agent by adding **multi-user session management**. The key enhancement is the ability to:

- **Store conversation history** in a SQLite database
- **Retrieve context** for each user based on their session ID
- **Enable contextual follow-up questions** like "What about Intel?" or "Which city is hotter?"

### Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Multi-User Conversational Agent                   │
├─────────────────────────────────────────────────────────────────────┤
│  User 1: john001          │  User 2: bond007                        │
│  ┌───────────────────┐    │  ┌───────────────────┐                  │
│  │ "Nvidia earnings?" │    │  │ "Weather in BLR?"  │                  │
│  │ "What about Intel?"│    │  │ "What about Mumbai?"│                  │
│  │ "Which is better?" │    │  │ "Which is hotter?"  │                  │
│  └───────────────────┘    │  └───────────────────┘                  │
│           │               │           │                             │
│           ▼               │           ▼                             │
│  ┌───────────────────────────────────────────────────────────┐      │
│  │                 SQLite Memory Database                     │      │
│  │  session_id: john001 → [message1, message2, message3]     │      │
│  │  session_id: bond007 → [message1, message2, message3]     │      │
│  └───────────────────────────────────────────────────────────┘      │
└─────────────────────────────────────────────────────────────────────┘
```

![](https://i.imgur.com/lHWqaT9.png)


---
## 🔧 Section 1: Environment Setup and Dependencies

First, let's install the required packages and set up our environment.

In [ ]:
# =============================================================================
# INSTALL DEPENDENCIES (Run once, then comment out)
# =============================================================================
# Uncomment these lines if you need to install the packages:

# !pip install langchain==0.3.14
# !pip install langchain-openai==0.3.0
# !pip install langchain-community==0.3.14
# !pip install markitdown

In [ ]:
# =============================================================================
# LOAD ENVIRONMENT VARIABLES
# =============================================================================
# This loads API keys from a .env file in the project directory.
# Your .env file should contain:
#   OPENAI_API_KEY=your_openai_key
#   TAVILY_API_KEY=your_tavily_key  
#   WEATHER_API_KEY=your_openweathermap_key
# =============================================================================

import os
from dotenv import load_dotenv, find_dotenv

# find_dotenv() searches for .env file in current and parent directories
load_dotenv(find_dotenv())

---
## 🛠️ Section 2: Create Custom Tools

We will create two custom tools that the agent can use:

1. **Web Search Tool** (`search_web_extract_info`): Searches the web using Tavily API and extracts content from search results
2. **Weather Tool** (`get_weather`): Fetches real-time weather data using OpenWeatherMap API

### Key Concepts:
- The `@tool` decorator converts a Python function into a LangChain tool
- Tools must have clear docstrings that describe their purpose (used by the LLM for tool selection)

![](https://i.imgur.com/TyPAYXE.png)

In [ ]:
# =============================================================================
# TOOL CREATION: Web Search and Weather Tools
# =============================================================================
# In LangChain, "tools" are functions that agents can invoke to interact with
# external systems (APIs, databases, web search, etc.).
#
# The @tool decorator converts a regular Python function into a LangChain-
# compatible tool. The docstring becomes the tool description that the LLM
# uses to decide WHEN to call this tool.
# =============================================================================

from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import os
from warnings import filterwarnings
from IPython.display import display, Markdown

filterwarnings('ignore')

# -----------------------------------------------------------------------------
# TOOL 1: Web Search and Information Extraction
# -----------------------------------------------------------------------------
# Initialize Tavily search with advanced settings for better quality results
tavily_tool = TavilySearchResults(
    max_results=5,           # Return top 5 search results
    search_depth='advanced', # Use advanced search for better relevance
    include_answer=False,    # Don't include AI-generated summary
    include_raw_content=True # Include raw HTML content from pages
)

# Configure HTTP session with browser-like headers to avoid bot detection
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})

# MarkItDown converts web pages to readable markdown text
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """
    Search the web for a query and extract useful information from the search links.
    
    This tool performs two main operations:
    1. Searches the web using Tavily API to find relevant URLs
    2. Extracts and converts content from each URL to readable text
    
    Args:
        query (str): The search query to find information about
        
    Returns:
        list: A list of extracted text content from relevant web pages
    """
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content

    # Process URLs in parallel for faster execution
    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                content = future.result(timeout=60)  # 60-second timeout
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


# -----------------------------------------------------------------------------
# TOOL 2: Weather Information Tool  
# -----------------------------------------------------------------------------
# This tool fetches real-time weather data using OpenWeatherMap API

@tool
def get_weather(query: str) -> dict:
    """
    Fetch current weather data for a specified location using OpenWeatherMap API.
    
    Args:
        query (str): The city name to get weather information for
        
    Returns:
        dict: Weather data including temperature, humidity, wind speed, etc.
              Returns "Weather Data Not Found" if the location is invalid
    """
    # Construct API URL with city name and API key
    # Note: The ',IN' suffix restricts search to India - modify as needed
    url = f"https://api.openweathermap.org/data/2.5/weather?q={query},IN&appid={os.getenv('WEATHER_API_KEY')}&units=metric"

    response = requests.get(url)
    data = response.json()
    
    # Validate response by checking if 'name' field exists
    if data.get("name"):
        return data
    else:
        return "Weather Data Not Found"

---
## 🤖 Section 3: Build the AI Agent

Now we'll create the agent with a **ReAct-style prompt** that guides the agent through:
- **Thought**: What should I do to answer this question?
- **Action**: Which tool should I use?
- **Observation**: What did the tool return?
- **Final Answer**: The complete response to the user

### Key Components:
1. **System Prompt** - Defines agent behavior and available tools
2. **Message Placeholders** - For conversation history and agent working memory
3. **Agent** - The decision-making component
4. **AgentExecutor** - The runtime that executes agent decisions

In [ ]:
# =============================================================================
# CREATE THE AGENT PROMPT TEMPLATE (ReAct Pattern)
# =============================================================================
# The prompt template defines how the agent should think and behave.
# We use the ReAct (Reasoning + Acting) pattern which structures the agent's
# thought process into: Thought → Action → Observation → (repeat) → Answer
#
# KEY ADDITION: The 'history' placeholder allows us to inject previous
# conversation messages, enabling contextual follow-up questions!
# =============================================================================

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

# Build the prompt template with multiple components:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),                                    # System instructions
        MessagesPlaceholder(variable_name="history", optional=True), # 🔑 Chat history for memory!
        ("human", "{query}"),                                      # User's current query
        MessagesPlaceholder(variable_name="agent_scratchpad"),     # Agent's working memory
    ]
)

print("Prompt template messages:")
prompt_template.messages

In [ ]:
# =============================================================================
# CREATE THE AGENT AND AGENT EXECUTOR
# =============================================================================
# The Agent is the "brain" that decides what actions to take.
# The AgentExecutor is the "body" that executes those actions.
#
# IMPORTANT: We pass 'chatgpt' (not 'chatgpt_with_tools') because
# create_tool_calling_agent handles .bind_tools() internally.
# =============================================================================

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI

# Initialize the LLM with temperature=0 for deterministic responses
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

# Define the tools available to the agent
tools = [search_web_extract_info, get_weather]

# Create the agent (LLM + Tools + Prompt)
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

# Create the AgentExecutor (runtime that executes agent decisions)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    early_stopping_method='force',  # Force stop if max iterations reached
    max_iterations=10               # Prevent infinite loops
)

print("✅ Agent and AgentExecutor created successfully!")

---
## 🧠 Section 4: Add Multi-User Conversation Memory

This is the **key differentiator** from the basic agent! We use `SQLChatMessageHistory` to:

1. **Store** each conversation in a SQLite database
2. **Retrieve** conversation history based on user/session ID
3. **Enable** contextual follow-up questions

### How It Works:

```python
# Each user gets their own conversation history:
session_id = 'john001'  # User identifier

# When user asks: "Summarize Nvidia's Q4 2024 earnings"
# → Agent searches web, returns summary, stores in DB under 'john001'

# Later, when user asks: "What about Intel?"
# → Agent retrieves history, sees context about earnings, searches for Intel

# Even later: "Which company is doing better?"
# → Agent uses full context to compare Nvidia vs Intel
```

### Why SQLite?
- **Persistence**: Conversations survive between sessions
- **Scalability**: Can handle multiple users simultaneously
- **Simplicity**: No external database server needed

In [ ]:
# =============================================================================
# OPTIONAL: Clear Previous Memory (Run only when you want a fresh start)
# =============================================================================
# This removes the SQLite database file that stores all conversation histories.
# Only run this if you want to reset ALL user conversations!

# !rm memory.db  # On Linux/Mac
# # Or on Windows: !del memory.db

In [ ]:
# =============================================================================
# CREATE MULTI-USER CONVERSATIONAL AGENT WITH SQL MEMORY
# =============================================================================
# This is where the magic happens! We wrap the AgentExecutor with:
# 1. SQLChatMessageHistory - Stores messages in SQLite database
# 2. RunnableWithMessageHistory - Automatically injects history into the agent
#
# Each user/session is identified by a unique 'session_id', and their
# conversations are stored separately in the database.
# =============================================================================

from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from IPython.display import display, Markdown

def get_session_history_db(session_id):
    """
    Retrieve conversation history from the SQLite database for a specific session.
    
    This function is called automatically by RunnableWithMessageHistory to:
    1. Load previous messages when starting a new conversation turn
    2. Save new messages after each interaction
    
    Args:
        session_id (str): Unique identifier for the user/session (e.g., 'john001')
        
    Returns:
        SQLChatMessageHistory: Object that manages message storage/retrieval
    """
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")


# Wrap the agent executor with message history capability
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,                 # The agent to wrap
    get_session_history_db,         # Function to get/create session history
    input_messages_key="query",     # Key for user input in the prompt
    history_messages_key="history", # Key for history in the prompt template
)


def chat_with_agent(prompt: str, session_id: str):
    """
    Send a message to the conversational agent and display the response.
    
    Args:
        prompt (str): The user's question or request
        session_id (str): Unique identifier for this user's conversation
    """
    response = agentic_chatbot.invoke(
        {"query": prompt},
        {'configurable': {'session_id': session_id}}  # Pass session ID for memory
    )
    display(Markdown(response['output']))


print("✅ Multi-user conversational agent ready!")

---
## 🧪 Section 5: Test Multi-User Conversations

Now let's test the agent with **two different users** to demonstrate:
1. Each user maintains their own conversation context
2. Follow-up questions work correctly within each session
3. Users don't see each other's conversation history

### Test Scenario 1: User 'john001' - Financial Research
We'll ask about company earnings and test contextual follow-ups.

In [ ]:
# =============================================================================
# USER 1: john001 - First Question (Nvidia Earnings)
# =============================================================================
# This is the initial question - no context exists yet.
# The agent will search the web and store this interaction in the database.

user_id = 'john001'
prompt = "Summarize the key points discussed in Nvidia's Q4 2024 earnings call"
chat_with_agent(prompt, user_id)

In [ ]:
# =============================================================================
# USER 1: john001 - Contextual Follow-up (Intel Earnings)
# =============================================================================
# 🔑 KEY POINT: The question "What about Intel?" only makes sense in context!
# Without memory, the agent wouldn't know we're asking about earnings.
# With memory, the agent understands we want Intel's Q4 2024 earnings.

prompt = "What about Intel?"
chat_with_agent(prompt, user_id)

In [ ]:
# =============================================================================
# USER 1: john001 - Comparative Analysis Using Context
# =============================================================================
# 🔑 This question requires understanding BOTH previous responses!
# The agent uses stored context about Nvidia and Intel to make a comparison.
# No new web search is needed - the information is already in memory.

prompt = "Which company seems to be doing better?"
chat_with_agent(prompt, user_id)

### Test Scenario 2: User 'bond007' - Weather Research
A different user with completely separate context.

In [ ]:
# =============================================================================
# USER 2: bond007 - Different User, Different Context (Weather)
# =============================================================================
# This is a completely different user with their own conversation history.
# They're asking about weather - unrelated to john001's financial queries.

user_id = 'bond007'
prompt = "how is the weather in Bangalore today? Show detailed statistics"
chat_with_agent(prompt, user_id)

In [ ]:
# =============================================================================
# USER 2: bond007 - Contextual Follow-up (Mumbai Weather)
# =============================================================================
# Again, "what about Mumbai?" only makes sense in context.
# The agent knows to fetch Mumbai's weather because of the previous query.

user_id = 'bond007'
prompt = "what about Mumbai?"
chat_with_agent(prompt, user_id)

In [ ]:
# =============================================================================
# USER 2: bond007 - Comparative Question Using Context
# =============================================================================
# Without memory, the agent wouldn't know which cities to compare!
# With memory, it understands we're comparing Bangalore vs Mumbai temperatures.

user_id = 'bond007'
prompt = "which city is hotter?"
chat_with_agent(prompt, user_id)

---
## 📝 Summary and Key Takeaways

### What We Learned:

1. **Session-Based Memory**: Using `SQLChatMessageHistory` to store conversations per user
2. **Contextual Understanding**: Agents can answer follow-up questions that reference previous context
3. **Multi-User Support**: Each user/session has isolated conversation history
4. **RunnableWithMessageHistory**: LangChain's way to add memory to any runnable

### Comparison: With vs Without Memory

| Question | Without Memory | With Memory |
|----------|---------------|-------------|
| "What about Intel?" | ❌ "Please clarify what you're asking about Intel" | ✅ Searches for Intel Q4 2024 earnings |
| "Which is better?" | ❌ "Which companies are you comparing?" | ✅ Compares Nvidia vs Intel based on context |
| "Which city is hotter?" | ❌ "Please specify the cities" | ✅ Compares Bangalore vs Mumbai |

### Next Steps:
- Explore **LangGraph** for more advanced agent architectures with stateful graphs
- Add **Redis** or **PostgreSQL** for production-scale memory storage
- Implement **conversation summarization** for very long conversations